In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import time
import urllib.request
import sys

def Image_Save(url,insta_id, datetime):
    print("before save")
    urllib.request.urlretrieve(url,'./src/saved_insta_'+str(insta_id)+"_"+str(datetime)+'.png')
    print("saved")

def InstagramUrlFromKeyword(browser, keyword, num_of_scroll):
    url = 'https://www.instagram.com/explore/tags/' + keyword +'/?hl=ko'
    browser.get(url)
    time.sleep(25)
    body = browser.find_element_by_tag_name('body')
    arr_href = []
    
    for i in range(num_of_scroll):
        time.sleep(0.5)
        body.send_keys(Keys.END)
        post = browser.find_elements_by_class_name('v1Nh3.kIKUG._bz0w')
        for item in post:
            try:
                href_element = item.find_element_by_css_selector('a')
                href_value = href_element.get_attribute('href')
                arr_href.append(href_value)
            except:
                pass
    arr_href = list(set(arr_href))
    return arr_href

def HashDataFromUrl(browser, url):
    browser.get(url)
    image_url = ""
    char = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!?#]')
    insta_id = browser.find_element_by_class_name('BrX75').text
    time.sleep(1)
    hash_text_area = browser.find_elements_by_class_name('C4VMK')
    hash_text = ""
    for hash_area in hash_text_area:
        text = hash_area.find_element_by_css_selector('span').text
        hash_text = hash_text + text
    hash_text = char.sub('', hash_text)
    views = ""
    heart = ""
    try:
        look_area = browser.find_element_by_class_name('EDfFK').text
        if "좋아요" in look_area:
            heart = look_area.split(" ")[-1][:-1].replace(",","")
            views = "-"
        else:
            heart = "-"
            views = look_area.split(" ")[-1][:-1].replace(",","")
    except:
        pass
    
    try:
        img_tag = browser.find_element_by_class_name('KL4Bh')
        img = img_tag.find_element_by_css_selector('img')
        src_set = img.get_attribute('srcset')
        image_url = src_set.split(',')[-1].split(" ")[0]
    except:
        image_url = ""
    
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    return insta_id, hash_text, heart, views, datetime, image_url

def HashTagFromInstagram(browser, keyword, num_of_scroll):
    arr_href = InstagramUrlFromKeyword(browser, keyword, num_of_scroll)
    insta_df = pd.DataFrame(columns=["Insta ID", "Content", "Heart", "Views", "Date"])
    for url in arr_href:
        try:
            insta_id, hash_text, heart, views, datetime, image_url = HashDataFromUrl(browser, url)
            insta_df = insta_df.append({"Insta ID" : insta_id, "Content": hash_text, "Heart" : heart, "Views" : views, "Date":datetime }, ignore_index=True)
            Image_Save(image_url,insta_id,  datetime)
            print("Image Url : ", image_url)
        except:
            print(sys.exc_info()[0])
            pass
    path = "instagram_crawling_" + keyword +".csv"
    insta_df.to_csv(path,mode='w',encoding='euc-kr')

In [ ]:
browser = webdriver.Chrome()
HashTagFromInstagram(browser, "namsantower", 1)

browser.close()
